<a href="https://colab.research.google.com/github/mateusribeirocampos/Calcd_Qt/blob/main/YOLOv3_Dec_seg_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!git clone https://github.com/AlexeyAB/darknet.git
%cd darknet
!make

In [ ]:
!sudo apt-get install tree

In [ ]:
!mkdir -p /content/coco/images
!mkdir -p /content/coco/annotations

In [5]:
# Baixar val2017 (1GB)
!wget -c http://images.cocodataset.org/zips/val2017.zip -P /content/coco/images

# Baixar test2017
!wget -c http://images.cocodataset.org/zips/test2017.zip -P /content/coco/images

--2025-02-15 21:52:57--  http://images.cocodataset.org/zips/val2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.216.209.81, 3.5.22.67, 16.182.73.233, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|52.216.209.81|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 815585330 (778M) [application/zip]
Saving to: ‘/content/coco/images/val2017.zip’

val2017.zip         100%[===================>] 777.80M  19.1MB/s    in 43s     

2025-02-15 21:53:41 (17.9 MB/s) - ‘/content/coco/images/val2017.zip’ saved [815585330/815585330]

--2025-02-15 21:53:41--  http://images.cocodataset.org/zips/test2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 54.231.195.89, 3.5.28.57, 3.5.29.71, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|54.231.195.89|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6646970404 (6.2G) [application/zip]
Saving to: ‘/content/coco/images/test2017.zip’

test

In [6]:
# Criar diretório para anotações
!mkdir -p /content/coco/annotations

# Baixar anotações (train + val)
!wget -c http://images.cocodataset.org/annotations/annotations_trainval2017.zip -P /content/coco/annotations

--2025-02-15 22:00:24--  http://images.cocodataset.org/annotations/annotations_trainval2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 16.15.217.139, 52.216.40.57, 52.216.250.196, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|16.15.217.139|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 252907541 (241M) [application/zip]
Saving to: ‘/content/coco/annotations/annotations_trainval2017.zip’

annotations_trainva 100%[===================>] 241.19M  20.5MB/s    in 13s     

2025-02-15 22:00:38 (18.2 MB/s) - ‘/content/coco/annotations/annotations_trainval2017.zip’ saved [252907541/252907541]



In [ ]:
!unzip /content/coco/images/val2017.zip -d /content/coco/images
!rm /content/coco/images/val2017.zip

!unzip /content/coco/images/test2017.zip -d /content/coco/images
!rm /content/coco/images/test2017.zip

In [8]:
# Baixar imagens de treino (18GB)
!wget -c http://images.cocodataset.org/zips/train2017.zip -P /content/coco/images

--2025-02-15 22:03:54--  http://images.cocodataset.org/zips/train2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.217.200.201, 52.217.203.89, 52.216.206.251, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|52.217.200.201|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19336861798 (18G) [application/zip]
Saving to: ‘/content/coco/images/train2017.zip’

train2017.zip       100%[===================>]  18.01G  20.0MB/s    in 16m 35s 

2025-02-15 22:20:29 (18.5 MB/s) - ‘/content/coco/images/train2017.zip’ saved [19336861798/19336861798]



In [ ]:
# Descompactar imagens
!unzip /content/coco/images/train2017.zip -d /content/coco/images
!rm /content/coco/images/train2017.zip

# Descompactar anotações
!unzip /content/coco/annotations/annotations_trainval2017.zip -d /content/coco/annotations
!rm /content/coco/annotations/annotations_trainval2017.zip

In [12]:
# Mova todos os JSONs para /content/coco/annotations/
!mv /content/coco/annotations/annotations/*.json /content/coco/annotations/
!rm -rf /content/coco/annotations/annotations  # Delete a subpasta vazia

In [13]:
!ls /content/coco/annotations/

captions_train2017.json  instances_train2017.json  person_keypoints_train2017.json
captions_val2017.json	 instances_val2017.json    person_keypoints_val2017.json


In [14]:
!tree -d /content/coco/

/content/coco/
├── annotations
└── images
    ├── test2017
    ├── train2017
    └── val2017

5 directories


In [17]:
!tree -d /content/coco/images/

/content/coco/images/
├── test2017
├── train2017
└── val2017

3 directories


In [15]:
import os
import shutil
from pycocotools.coco import COCO

def process_coco_split(splits: list, categories: list, max_samples_per_split: int = None):
    """
    Processa splits do COCO (train, val, test), filtrando por categorias e convertendo para YOLO.
    """
    # Criar diretório principal
    os.makedirs('/content/coco/filtered/', exist_ok=True)

    # Criar arquivo de classes
    with open('/content/coco/filtered/obj.names', 'w') as f:
        for cat in categories:
            f.write(cat + '\n')

    for split in splits:
        print(f"\nProcessando split: {split}")

        # Caminhos das imagens
        img_source_dir = f'/content/coco/images/{split}2017'
        img_dest_dir = f'/content/coco/filtered/{split}/images'
        os.makedirs(img_dest_dir, exist_ok=True)

        # Processar apenas train/val (test não tem anotações)
        if split != 'test':
            # Carregar anotações
            coco = COCO(f'/content/coco/annotations/instances_{split}2017.json')
            cat_ids = coco.getCatIds(catNms=categories)
            img_ids = coco.getImgIds(catIds=cat_ids)

            # Limitar amostras se especificado
            if max_samples_per_split:
                img_ids = img_ids[:max_samples_per_split]

            images = coco.loadImgs(img_ids)

            # Copiar imagens
            for img in images:
                src = os.path.join(img_source_dir, img['file_name'])
                dst = os.path.join(img_dest_dir, img['file_name'])
                shutil.copy(src, dst)

            # Criar diretório de labels
            ann_dest_dir = f'/content/coco/filtered/{split}/labels'
            os.makedirs(ann_dest_dir, exist_ok=True)

            # Converter anotações para YOLO
            for img in images:
                ann_ids = coco.getAnnIds(imgIds=img['id'], catIds=cat_ids)
                annotations = coco.loadAnns(ann_ids)

                label_path = os.path.join(ann_dest_dir, img['file_name'].replace('.jpg', '.txt'))
                with open(label_path, 'w') as f:
                    for ann in annotations:
                        x, y, w, h = ann['bbox']
                        img_w, img_h = img['width'], img['height']

                        cx = (x + w/2) / img_w
                        cy = (y + h/2) / img_h
                        nw = w / img_w
                        nh = h / img_h

                        class_id = categories.index(coco.loadCats(ann['category_id'])[0]['name'])
                        f.write(f"{class_id} {cx:.6f} {cy:.6f} {nw:.6f} {nh:.6f}\n")

        # Processar test (sem anotações)
        else:
            # Listar todas as imagens do diretório test2017
            images = [{'file_name': f} for f in os.listdir(img_source_dir)]
            if max_samples_per_split:
                images = images[:max_samples_per_split]

            # Copiar imagens
            for img in images:
                src = os.path.join(img_source_dir, img['file_name'])
                dst = os.path.join(img_dest_dir, img['file_name'])
                shutil.copy(src, dst)

# Uso
process_coco_split(
    splits=['train', 'val', 'test'],
    categories=['boat', 'person'],
    max_samples_per_split=100
)


Processando split: train
loading annotations into memory...
Done (t=20.65s)
creating index...
index created!

Processando split: val
loading annotations into memory...
Done (t=0.66s)
creating index...
index created!

Processando split: test


In [16]:
!wget https://pjreddie.com/media/files/yolov3.weights

--2025-02-15 22:26:42--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 162.0.215.52
Connecting to pjreddie.com (pjreddie.com)|162.0.215.52|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘yolov3.weights’

yolov3.weights      100%[===================>] 236.52M  18.9MB/s    in 14s     

2025-02-15 22:26:56 (17.2 MB/s) - ‘yolov3.weights’ saved [248007048/248007048]



In [18]:
!ls

3rdparty	darknet			Dockerfile.cpu	       Makefile       scripts
backup		DarknetConfig.cmake.in	Dockerfile.gpu	       net_cam_v3.sh  src
build		darknet_images.py	image_yolov3.sh        net_cam_v4.sh  vcpkg.json
build.ps1	darknet.py		image_yolov4.sh        obj	      video_yolov3.sh
cfg		darknet_video.py	include		       package.xml    video_yolov4.sh
cmake		data			json_mjpeg_streams.sh  README.md      yolov3.weights
CMakeLists.txt	docker-compose.yml	LICENSE		       results
